<a href="https://colab.research.google.com/github/nonafalaki/Multimodal-Framework/blob/main/houseprice_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import os
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cols = ["bedrooms", "bathrooms","area", "zipcode", "price"]
inputpath = '/content/drive/MyDrive/HousesDataset/HousesInfo.txt'
df_houses = pd.read_csv(inputpath, sep=" ", header=None, names=cols)

In [ ]:
df_houses

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226
...,...,...,...,...,...
530,5,2.0,2066,94531,399900
531,4,3.5,9536,94531,460000
532,3,2.0,2014,94531,407000
533,4,3.0,2312,94531,419000


In [40]:
from pandas.core.groupby import base
import os
import glob
import cv2
images = []
Path='/content/drive/MyDrive/HousesDataset'
for i in df_houses.index.values:
  basePath = os.path.sep.join([Path, "{}_*".format(i + 1)])
  housePaths = sorted(list(glob.glob(basePath)))
  inputImages = []
  outputImage = np.zeros((128, 128, 3), dtype="uint8")
  for housePath in housePaths:
    image = cv2.imread(housePath)
    image = cv2.resize(image, (64, 64))
    inputImages.append(image)

  outputImage[0:64, 0:64] = inputImages[0]
  outputImage[0:64, 64:128] = inputImages[1]
  outputImage[64:128, 64:128] = inputImages[2]
  outputImage[64:128, 0:64] = inputImages[3]
	# add the tiled image to our set of images the network will be
	# trained on
  images.append(outputImage)

In [ ]:
def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
 	# determine (1) the unique zip codes and (2) the number of data
	# points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
	# loop over each of the unique zip codes and their corresponding
	# count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
	# return the data frame
	return df

In [34]:
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
def process_house_features(df, train, test):
	con_features = ["bedrooms", "bathrooms", "area"]
	minmax = MinMaxScaler()
	trainContinuous = minmax.fit_transform(train[con_features])
	testContinuous = minmax.transform(test[con_features])

	Binarizer = LabelBinarizer().fit(df["zipcode"])
	trainCategorical = Binarizer.transform(train["zipcode"])
	testCategorical = Binarizer.transform(test["zipcode"])

	trainX = np.hstack([trainCategorical, trainContinuous])
	testX = np.hstack([testCategorical, testContinuous])
	return (trainX, testX)